In [ ]:
import os
import time

# --- 1. Sirf is line ko change karein ---
# 🔴 Pehli baar 'images1_archive.tar.gz' likhein
#    Doosri baar 'images2_archive.tar.gz' likhein, etc.
ARCHIVE_FILE_NAME = "images_archive.tar.gz"

# --- 2. In paths ko hamesha same rehne dein ---
DRIVE_BASE_PATH = "/content/drive/MyDrive/Pneumonia_Advanced_Project"
# Yeh hamesha same folder rahega taake images jama (collect) ho sakein
LOCAL_IMAGE_DIR = "/content/local_images"

# --- 3. Baqi code ab automatically kaam karega ---
DRIVE_TAR_PATH = os.path.join(DRIVE_BASE_PATH, ARCHIVE_FILE_NAME)
LOCAL_TAR_PATH = f"/content/{ARCHIVE_FILE_NAME}"

# Yeh line check karti hai ke folder hai ya nahi. Agar hai, to usay istemaal karti hai.
os.makedirs(LOCAL_IMAGE_DIR, exist_ok=True)
print(f"--- Setup Shuru ---")
print(f"Target Folder: {LOCAL_IMAGE_DIR}")

# --- FAST COPY ---
print(f"Copying {ARCHIVE_FILE_NAME}...")
start_time = time.time()
!cp "{DRIVE_TAR_PATH}" "{LOCAL_TAR_PATH}"
print(f"Copy complete in {(time.time() - start_time):.2f} seconds.")

# --- FAST UNPACK (ADD) ---
print(f"Extracting and ADDING images to {LOCAL_IMAGE_DIR}...")
start_time = time.time()
# Yeh command purani files ko delete nahi karta, sirf nayi files add karta hai
!tar -xzf "{LOCAL_TAR_PATH}" -C "{LOCAL_IMAGE_DIR}"
print(f"Extraction complete in {(time.time() - start_time):.2f} seconds.")

print(f"\n✅ --- READY TO TRAIN! Images are in {LOCAL_IMAGE_DIR} ---")

--- Setup Shuru ---
Target Folder: /content/local_images
Copying images_archive.tar.gz...
^C
Copy complete in 4.06 seconds.
Extracting and ADDING images to /content/local_images...
^C
Extraction complete in 1.56 seconds.

✅ --- READY TO TRAIN! Images are in /content/local_images ---


In [ ]:
import os

# The path where you extracted the images
LOCAL_IMAGE_DIR = "/content/local_images"

try:
    file_count = len(os.listdir(LOCAL_IMAGE_DIR))
    print(f"✅ Success! Found {file_count} images in '{LOCAL_IMAGE_DIR}'.")
except FileNotFoundError:
    print(f"❌ Error: Could not find the directory '{LOCAL_IMAGE_DIR}'.")
except Exception as e:
    print(f"An error occurred: {e}")

✅ Success! Found 30051 images in '/content/local_images'.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

# --- Configuration ---
IMG_WIDTH, IMG_HEIGHT = 224, 224
BATCH_SIZE = 32
EPOCHS_HEAD = 10       # Epochs for training the head
EPOCHS_FINE_TUNE = 20  # Epochs for fine-tuning
LEARNING_RATE_HEAD = 1e-3
LEARNING_RATE_FINE_TUNE = 1e-5
IMAGE_DIR = '/content/local_images'   # The folder with all 30k+ images
MODEL_SAVE_PATH = 'v2_multiclass_model.keras' # Using .keras format

# --- THE NEW CLASS WEIGHTS DICTIONARY (from your output) ---
CLASS_WEIGHTS_DICT = {
    0: 1.3844487463126844,
    1: 0.8326957196717676,
    2: 0.33555644927050515,
    3: 2.2368930592791183,
    4: 1.961664054336468,
    5: 7.151666666666666
}
# These correspond to your indices:
# {'COVID-19': 0, 'Lung Opacity': 1, 'Normal': 2, 'Pneumonia (Bacterial)': 3, 'Pneumonia (Viral)': 4, 'Tuberculosis': 5}
CLASS_NAMES = ['COVID-19', 'Lung Opacity', 'Normal', 'Pneumonia (Bacterial)', 'Pneumonia (Viral)', 'Tuberculosis']
NUM_CLASSES = len(CLASS_NAMES)

# --- 1. Load Full Dataset ---
print("Loading master CSV...")
# 🔴 KEY CHANGE 1: Use the full master CSV, not a balanced subset
try:
    df = pd.read_csv('/content/master_tf_clean.csv') # Make sure this filename is correct
    # Add the full file path to the 'filename' column
    df['filepath'] = df['filename'].apply(lambda x: os.path.join(IMAGE_DIR, x))
    # Map class names to integers (indices)
    class_indices = {name: i for i, name in enumerate(CLASS_NAMES)}
    df['label_idx'] = df['label'].map(class_indices)
except Exception as e:
    print(f"Error loading CSV or mapping paths: {e}")
    exit()

print(f"Loaded {len(df)} image records.")

# --- 2. Split Data ---
print("Splitting data into train, validation, and test sets...")
# First, split into train+val (90%) and test (10%)
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=42, stratify=df['label'])
# Then, split train+val into train (80% of original) and val (10% of original)
train_df, val_df = train_test_split(train_val_df, test_size=(len(test_df)/len(train_val_df)), random_state=42, stratify=train_val_df['label'])

print(f"Training images: {len(train_df)}")
print(f"Validation images: {len(val_df)}")
print(f"Test images: {len(test_df)}")

# --- 3. Create tf.data Pipeline ---
print("Building tf.data pipelines...")

def load_and_preprocess(filepath, label):
    img = tf.io.read_file(filepath)

    # --- YEH HAI ASAL CHANGE ---
    # decode_image har format (PNG, JPEG, GIF) ko handle kar leta hai
    # expand_animations=False GIF files ko expand honay se rokta hai
    img = tf.image.decode_image(img, channels=3, expand_animations=False)
    # --------------------------

    img = tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])
    img = tf.keras.applications.densenet.preprocess_input(img)
    return img, label

def build_dataset(df, augment=False):
    # Keras GPU-powered augmentation layers
    augmentation_layers = tf.keras.Sequential([
        tf.keras.layers.RandomFlip("horizontal"),
        tf.keras.layers.RandomRotation(0.1),
        tf.keras.layers.RandomZoom(0.1),
    ])

    dataset = tf.data.Dataset.from_tensor_slices((df['filepath'].values, df['label_idx'].values))
    dataset = dataset.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

    if augment:
        dataset = dataset.map(lambda x, y: (augmentation_layers(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)

    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = build_dataset(train_df, augment=True)
val_dataset = build_dataset(val_df)
test_dataset = build_dataset(test_df) # We will use this at the very end

# --- 4. Build Model ---
print("Building DenseNet-121 model...")

def build_densenet_model():
    base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    # Freeze the base model
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

model = build_densenet_model()

# --- 5. Stage 1: Train the Head ---
print("--- STAGE 1: Training the new head ---")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_HEAD),
    loss='sparse_categorical_crossentropy', # Use sparse_categorical_crossentropy since labels are integers
    metrics=['accuracy']
)

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_loss', save_best_only=True)

history_head = model.fit(
    train_dataset,
    epochs=EPOCHS_HEAD,
    validation_data=val_dataset,
    callbacks=[early_stopping, model_checkpoint],
    # 🔴 KEY CHANGE 2: Pass the class weights dictionary here!
    class_weight=CLASS_WEIGHTS_DICT
)

print("Head training complete. Best weights loaded.")

# --- 6. Stage 2: Fine-Tuning ---
print("--- STAGE 2: Fine-tuning the full model ---")
# Unfreeze the base model (or parts of it)
model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE_FINE_TUNE), # Use a very low learning rate
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training from where we left off
history_fine_tune = model.fit(
    train_dataset,
    epochs=EPOCHS_FINE_TUNE, # You can use a new number of epochs
    validation_data=val_dataset,
    callbacks=[early_stopping, model_checkpoint], # Re-using the same callbacks
    # 🔴 KEY CHANGE 2 (Again): Pass the weights for fine-tuning as well!
    class_weight=CLASS_WEIGHTS_DICT
)

print("--- Training Complete! ---")
print(f"Model saved to {MODEL_SAVE_PATH}")

# --- 7. Final Evaluation ---
print("Evaluating model on the hold-out test set...")
# Load the best model saved by ModelCheckpoint
model.load_weights(MODEL_SAVE_PATH)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

# --- IMPORTANT: Generate Classification Report ---
from sklearn.metrics import classification_report, confusion_matrix

print("Generating classification report and confusion matrix...")
# Get all true labels and predictions from the test set
y_true = []
y_pred_probs = []

for images, labels in test_dataset:
    y_true.extend(labels.numpy())
    y_pred_probs.extend(model.predict(images))

y_pred = np.argmax(y_pred_probs, axis=1)

print("\n--- Confusion Matrix ---")
print(confusion_matrix(y_true, y_pred))

print("\n--- Classification Report ---")
print(classification_report(y_true, y_pred, target_names=CLASS_NAMES))

Loading master CSV...
Loaded 30036 image records.
Splitting data into train, validation, and test sets...
Training images: 24028
Validation images: 3004
Test images: 3004
Building tf.data pipelines...
Building DenseNet-121 model...
--- STAGE 1: Training the new head ---
Epoch 1/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 444s 557ms/step - accuracy: 0.5396 - loss: 1.1290 - val_accuracy: 0.7873 - val_loss: 0.5974
Epoch 2/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 380s 506ms/step - accuracy: 0.6996 - loss: 0.6934 - val_accuracy: 0.7477 - val_loss: 0.6430
Epoch 3/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 375s 496ms/step - accuracy: 0.7233 - loss: 0.6308 - val_accuracy: 0.8033 - val_loss: 0.5209
Epoch 4/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 370s 492ms/step - accuracy: 0.7287 - loss: 0.6238 - val_accuracy: 0.8132 - val_loss: 0.5304
Epoch 5/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 383s 494ms/step - accuracy: 0.7385 - loss: 0.5907 - val_accuracy: 0.7889 - val_loss: 0.5675
Epoch 6/10
751/751 ━━━━━━━━━━━━━━━━━━━━ 374s 498ms/step - accuracy: 0

In [ ]:
import os
import pandas as pd
from PIL import Image

# --- 1. Paths (Inhein check kar lein) ---
# Wohi path jahan aapne images ko extract kiya hai
LOCAL_IMAGE_DIR = "/content/local_images"
# Aapki asal CSV file
MASTER_CSV_PATH = "/content/drive/MyDrive/Pneumonia_Advanced_Project/2_processed_data/master_labels.csv" # 🔴 Is file ka naam theek karein

print(f"Loading '{MASTER_CSV_PATH}'...")
try:
    df = pd.read_csv(MASTER_CSV_PATH)
except FileNotFoundError:
    print(f"❌ ERROR: '{MASTER_CSV_PATH}' file nahi mili. Please check the filename.")
    raise

print(f"Loaded {len(df)} records. Ab har file ko check kiya ja raha hai...")

# --- 2. Files ko check karein aur clean list banayein ---
good_files_list = []
bad_files_count = 0

# tqdm progress bar ke liye
from tqdm.notebook import tqdm

for index, row in tqdm(df.iterrows(), total=len(df), desc="Checking images"):
    # CSV se file ka naam aur label lein
    filename = row['filename'] # 🔴 Check karein ke column ka naam 'filename' hai
    label = row['label']     # 🔴 Check karein ke column ka naam 'label' hai

    # Poora path banayein
    file_path = os.path.join(LOCAL_IMAGE_DIR, filename)

    # Check karein ke file local disk par maujood hai
    if not os.path.exists(file_path):
        bad_files_count += 1
        continue # File missing hai, isay chhor dein

    # Check karein ke file corrupt to nahi
    try:
        with Image.open(file_path) as img:
            img.verify() # File ko poora parhe bina uski integrity check karein

        # Agar sab theek hai, to isay 'good' list mein add karein
        good_files_list.append({'filename': filename, 'label': label})

    except Exception as e:
        # File corrupt hai
        print(f"\nCorrupt file dhoondi gayi (isay ignore kiya jayega): {filename} - Error: {e}")
        bad_files_count += 1

# --- 3. Nayi clean CSV banayein ---
CLEAN_CSV_PATH = "master_clean.csv"
clean_df = pd.DataFrame(good_files_list)
clean_df.to_csv(CLEAN_CSV_PATH, index=False)

print("\n--- Cleaning Complete! ---")
print(f"Original records: {len(df)}")
print(f"Bad or Missing files: {bad_files_count}")
print(f"Good files found: {len(clean_df)}")
print(f"✅ Nayi clean file '{CLEAN_CSV_PATH}' ban gayi hai.")

Loading '/content/drive/MyDrive/Pneumonia_Advanced_Project/2_processed_data/master_labels.csv'...
Loaded 30037 records. Ab har file ko check kiya ja raha hai...


Checking images:   0%|          | 0/30037 [00:00<?, ?it/s]


--- Cleaning Complete! ---
Original records: 30037
Bad or Missing files: 0
Good files found: 30037
✅ Nayi clean file 'master_clean.csv' ban gayi hai.
